In [1]:
import tkinter as tk
import random
import math
import numpy as np

In [2]:
class Brain():

    def __init__(self,botp):
        # self is needed to show that an object belongs to a class 
        #Self represents the instance of the class. 
        #By using the “self” we can access the attributes and methods of the class in Python
        self.bot = botp
        self.turningCount = 0
        self.movingCount = random.randrange(50,100)
        self.currentlyTurning = False
        self.map = np.zeros((10,10))

    # modify this to change the robot's behaviour
    def thinkAndAct(self, lightL, lightR, chargerL, chargerR, x, y, sl, sr, battery):
        newX = None
        newY = None
        
        # wandering behaviour
        if self.currentlyTurning==True:
            speedLeft = -2.0
            speedRight = 2.0
            self.turningCount -= 1
        else:
            speedLeft = 5.0
            speedRight = 5.0
            self.movingCount -= 1
        if self.movingCount==0 and not self.currentlyTurning:
            self.turningCount = random.randrange(20,40)
            self.currentlyTurning = True
        if self.turningCount==0 and self.currentlyTurning:
            self.movingCount = random.randrange(50,100)
            self.currentlyTurning = False

        #battery - these are later so they have priority
        if battery<600:
            if chargerR>chargerL:
                speedLeft = 2.0
                speedRight = -2.0
            elif chargerR<chargerL:
                speedLeft = -2.0
                speedRight = 2.0
            if abs(chargerR-chargerL)<chargerL*0.1: #approximately the same
                speedLeft = 5.0
                speedRight = 5.0
        if chargerL+chargerR>200 and battery<1000:
            speedLeft = 0.0
            speedRight = 0.0

        #toroidal geometry
        if x>1000:
            newX = 0
        if x<0:
            newX = 1000
        if y>1000:
            newY = 0
        if y<0:
            newY = 1000
            
        def updateMap(self):
            #to calculate which square of the map we are in , need to divide the screen into 10 by 10 square grid to get the map posiiton of the bot 
            xMapPosition = int(math.floor(self.x/100))
            yMapPosition = int(math.floor(self.y/100))
            self.map = np.ones((x,y))
            
            

        return speedLeft, speedRight, newX, newY

In [37]:
class Bot():

    def __init__(self,namep):
        self.name = namep
        self.x = random.randint(100,600)
        self.y = random.randint(100,600)
        self.theta = random.uniform(0.0,2.0*math.pi)
        # self.theta = 0
        self.ll = 60  # axle width
        self.sl = 0.0
        self.sr = 0.0
        self.battery = 1000

    def thinkAndAct(self, agents, passiveObjects):
        lightL, lightR = self.senseLight(passiveObjects)
        chargerL, chargerR = self.senseChargers(passiveObjects)
        self.sl, self.sr, xx, yy = self.brain.thinkAndAct\
            (lightL, lightR, chargerL, chargerR, self.x, self.y, self.sl, self.sr, self.battery)
        if xx != None:
            self.x = xx
        if yy != None:
            self.y = yy
        
    def setBrain(self,brainp):
        self.brain = brainp

    #returns the output from polling the light sensors
    def senseLight(self, passiveObjects):
        lightL = 0.0
        lightR = 0.0
        for pp in passiveObjects:
            if isinstance(pp,Lamp):
                lx,ly = pp.getLocation()
                distanceL = math.sqrt( (lx-self.sensorPositions[0])*(lx-self.sensorPositions[0]) + \
                                       (ly-self.sensorPositions[1])*(ly-self.sensorPositions[1]) )
                distanceR = math.sqrt( (lx-self.sensorPositions[2])*(lx-self.sensorPositions[2]) + \
                                       (ly-self.sensorPositions[3])*(ly-self.sensorPositions[3]) )
                lightL += 200000/(distanceL*distanceL)
                lightR += 200000/(distanceR*distanceR)
        return lightL, lightR

    #returns sensors values that detect chargers
    def senseChargers(self, passiveObjects):
        chargerL = 0.0
        chargerR = 0.0
        for pp in passiveObjects:
            if isinstance(pp,Charger):
                lx,ly = pp.getLocation()
                distanceL = math.sqrt( (lx-self.sensorPositions[0])*(lx-self.sensorPositions[0]) + \
                                       (ly-self.sensorPositions[1])*(ly-self.sensorPositions[1]) )
                distanceR = math.sqrt( (lx-self.sensorPositions[2])*(lx-self.sensorPositions[2]) + \
                                       (ly-self.sensorPositions[3])*(ly-self.sensorPositions[3]) )
                chargerL += 200000/(distanceL*distanceL)
                chargerR += 200000/(distanceR*distanceR)
        return chargerL, chargerR

    def distanceTo(self,obj):
        xx,yy = obj.getLocation()
        return math.sqrt( math.pow(self.x-xx,2) + math.pow(self.y-yy,2) )

    # what happens at each timestep
    def update(self,canvas,passiveObjects,dt):
        # for now, the only thing that changes is that the robot moves
        #   (using the current settings of self.sl and self.sr)
        self.battery -= 1
        for rr in passiveObjects:
            if isinstance(rr,Charger) and self.distanceTo(rr)<80:
                self.battery += 10
        if self.battery<=0:
            self.battery = 0
        self.move(canvas,dt)

    # draws the robot at its current position
    def draw(self,canvas):
        points = [ (self.x + 30*math.sin(self.theta)) - 30*math.sin((math.pi/2.0)-self.theta), \
                   (self.y - 30*math.cos(self.theta)) - 30*math.cos((math.pi/2.0)-self.theta), \
                   (self.x - 30*math.sin(self.theta)) - 30*math.sin((math.pi/2.0)-self.theta), \
                   (self.y + 30*math.cos(self.theta)) - 30*math.cos((math.pi/2.0)-self.theta), \
                   (self.x - 30*math.sin(self.theta)) + 30*math.sin((math.pi/2.0)-self.theta), \
                   (self.y + 30*math.cos(self.theta)) + 30*math.cos((math.pi/2.0)-self.theta), \
                   (self.x + 30*math.sin(self.theta)) + 30*math.sin((math.pi/2.0)-self.theta), \
                   (self.y - 30*math.cos(self.theta)) + 30*math.cos((math.pi/2.0)-self.theta)  \
                ]
        canvas.create_polygon(points, fill="blue", tags=self.name)

        self.sensorPositions = [ (self.x + 20*math.sin(self.theta)) + 30*math.sin((math.pi/2.0)-self.theta), \
                                 (self.y - 20*math.cos(self.theta)) + 30*math.cos((math.pi/2.0)-self.theta), \
                                 (self.x - 20*math.sin(self.theta)) + 30*math.sin((math.pi/2.0)-self.theta), \
                                 (self.y + 20*math.cos(self.theta)) + 30*math.cos((math.pi/2.0)-self.theta)  \
                            ]
    
        centre1PosX = self.x 
        centre1PosY = self.y
        canvas.create_oval(centre1PosX-16,centre1PosY-16,\
                           centre1PosX+16,centre1PosY+16,\
                           fill="gold",tags=self.name)
        canvas.create_text(self.x,self.y,text=str(self.battery),tags=self.name)

        wheel1PosX = self.x - 30*math.sin(self.theta)
        wheel1PosY = self.y + 30*math.cos(self.theta)
        canvas.create_oval(wheel1PosX-3,wheel1PosY-3,\
                                         wheel1PosX+3,wheel1PosY+3,\
                                         fill="red",tags=self.name)

        wheel2PosX = self.x + 30*math.sin(self.theta)
        wheel2PosY = self.y - 30*math.cos(self.theta)
        canvas.create_oval(wheel2PosX-3,wheel2PosY-3,\
                                         wheel2PosX+3,wheel2PosY+3,\
                                         fill="green",tags=self.name)

        sensor1PosX = self.sensorPositions[0]
        sensor1PosY = self.sensorPositions[1]
        sensor2PosX = self.sensorPositions[2]
        sensor2PosY = self.sensorPositions[3]
        canvas.create_oval(sensor1PosX-3,sensor1PosY-3, \
                           sensor1PosX+3,sensor1PosY+3, \
                           fill="yellow",tags=self.name)
        canvas.create_oval(sensor2PosX-3,sensor2PosY-3, \
                           sensor2PosX+3,sensor2PosY+3, \
                           fill="yellow",tags=self.name)

    # handles the physics of the movement
    # cf. Dudek and Jenkin, Computational Principles of Mobile Robotics
    def move(self,canvas,dt):
        print(self.battery)
        if self.battery==0:
            self.sl = 0
            self.sl = 0
        if self.sl==self.sr:
            R = 0
        else:
            R = (self.ll/2.0)*((self.sr+self.sl)/(self.sl-self.sr))
        omega = (self.sl-self.sr)/self.ll
        ICCx = self.x-R*math.sin(self.theta) #instantaneous centre of curvature
        ICCy = self.y+R*math.cos(self.theta)
        m = np.matrix( [ [math.cos(omega*dt), -math.sin(omega*dt), 0], \
                        [math.sin(omega*dt), math.cos(omega*dt), 0],  \
                        [0,0,1] ] )
        v1 = np.matrix([[self.x-ICCx],[self.y-ICCy],[self.theta]])
        v2 = np.matrix([[ICCx],[ICCy],[omega*dt]])
        newv = np.add(np.dot(m,v1),v2)
        newX = newv.item(0)
        newY = newv.item(1)
        newTheta = newv.item(2)
        newTheta = newTheta%(2.0*math.pi) #make sure angle doesn't go outside [0.0,2*pi)
        self.x = newX
        self.y = newY
        self.theta = newTheta        
        if self.sl==self.sr: # straight line movement
            self.x += self.sr*math.cos(self.theta) #sr wlog
            self.y += self.sr*math.sin(self.theta)
        canvas.delete(self.name)
        self.draw(canvas)
        updateMap()
       

    def collectDirt(self, canvas, passiveObjects, count):
        toDelete = []
        for idx,rr in enumerate(passiveObjects):
            if isinstance(rr,Dirt):
                if self.distanceTo(rr)<30:
                    canvas.delete(rr.name)
                    toDelete.append(idx)
                    #call item collected 
                    count.itemCollected(count.dirtCollected)
                    print("dirt collected",count.dirtCollected)
            
        for ii in sorted(toDelete,reverse=True):
            del passiveObjects[ii]
        return passiveObjects 
    
    def drawMap(self):
        #create an instance of the class brain in order to access the function
        brain_instance = Brain()
        
        #two nested for loops to extract the value of x and y 
        for row in 
            for 
                if x==1 && y==1 :
                    self.canvas.create_rectangle(100*xx,100*yy,100*xx+100,100*yy+100, fill="pink", width=0, tags="map")
            
        
        

SyntaxError: incomplete input (3446238575.py, line 172)

In [4]:
class Lamp():
    def __init__(self,namep):
        self.centreX = random.randint(100,600)
        self.centreY = random.randint(100,600)
        self.name = namep
        
    def draw(self,canvas):
        body = canvas.create_oval(self.centreX-10,self.centreY-10, \
                                  self.centreX+10,self.centreY+10, \
                                  fill="yellow",tags=self.name)

    def getLocation(self):
        return self.centreX, self.centreY


In [5]:
class Charger():
    def __init__(self,namep):
        self.centreX = random.randint(100,600)
        self.centreY = random.randint(100,600)
        self.name = namep
        
    def draw(self,canvas):
        body = canvas.create_oval(self.centreX-10,self.centreY-10, \
                                  self.centreX+10,self.centreY+10, \
                                  fill="red",tags=self.name)

    def getLocation(self):
        return self.centreX, self.centreY

In [6]:
class WiFiHub:
    def __init__(self,namep,xp,yp):
        self.centreX = xp
        self.centreY = yp
        self.name = namep
        
    def draw(self,canvas):
        body = canvas.create_oval(self.centreX-10,self.centreY-10, \
                                  self.centreX+10,self.centreY+10, \
                                  fill="purple",tags=self.name)

    def getLocation(self):
        return self.centreX, self.centreY

In [7]:
class Dirt:
    def __init__(self,namep):
        self.centreX = random.randint(100,600)
        self.centreY = random.randint(100,600)
        self.name = namep

    def draw(self,canvas):
        body = canvas.create_oval(self.centreX-1,self.centreY-1, \
                                  self.centreX+1,self.centreY+1, \
                                  fill="grey",tags=self.name)

    def getLocation(self):
        return self.centreX, self.centreY

In [30]:
class Counter: 
    
    def __init__(self): 
        
        #class is initialised with the variable dirtCollected set to 0 
        self.dirtCollected = 0
    
    def itemCollected(self,canvas): 
        #increase the count of dirt collected by 1 
        self.dirtCollected += 1
        
        #canvas.create_text(50,50,anchor="w",\
                           #text="Dirt collected: "+str(self.dirtCollected),\
                          # tags="dirtCount")
        #canvas.delete("dirtCount")
        
        
    

In [31]:
def initialise(window):
    window.resizable(False,False)
    canvas = tk.Canvas(window,width=700,height=700)
    canvas.pack()
    return canvas

In [32]:
def buttonClicked(x,y,agents):
    for rr in agents:
        if isinstance(rr,Bot):
            rr.x = x
            rr.y = y

In [33]:
def createObjects(canvas,noOfBots=2,noOfLights=2,amountOfDirt=300):
    agents = []
    passiveObjects = []
    for i in range(0,noOfBots):
        bot = Bot("Bot"+str(i))
        brain = Brain(bot)
        bot.setBrain(brain)
        agents.append(bot)
        bot.draw(canvas)

    for i in range(0,noOfLights):
        lamp = Lamp("Lamp"+str(i))
        passiveObjects.append(lamp)
        lamp.draw(canvas)

    charger = Charger("Charger"+str(i))
    passiveObjects.append(charger)
    charger.draw(canvas)
    
    hub1 = WiFiHub("Hub1",950,50)
    passiveObjects.append(hub1)
    hub1.draw(canvas)
    hub2 = WiFiHub("Hub2",50,500)
    passiveObjects.append(hub2)
    hub2.draw(canvas)

    for i in range(0,amountOfDirt):
        dirt = Dirt("Dirt"+str(i))
        passiveObjects.append(dirt)
        dirt.draw(canvas)
    count = Counter()
    canvas.bind( "<Button-1>", lambda event: buttonClicked(event.x,event.y,agents) )
    return agents, passiveObjects, count



In [34]:
def moveIt(canvas,agents,passiveObjects,count):
    for rr in agents:
        rr.thinkAndAct(agents,passiveObjects)
        rr.update(canvas,passiveObjects,1.0)
        passiveObjects = rr.collectDirt(canvas,passiveObjects,count)
    canvas.after(50,moveIt,canvas,agents,passiveObjects, count)

In [35]:
def main():
    window = tk.Tk()
    canvas = initialise(window)
    agents, passiveObjects, count = createObjects(canvas,noOfBots=2,noOfLights=0,amountOfDirt=300)
    #collectDirt(registryActives, registryPassives)
    moveIt(canvas,agents,passiveObjects,count)
    window.mainloop()

In [36]:
main()

999
dirt collected 1
dirt collected 2
999
dirt collected 3
dirt collected 4
998
998
997
997
996
dirt collected 5
996
995
995
dirt collected 6
994
994
993
993
dirt collected 7
992
dirt collected 8
992
dirt collected 9
dirt collected 10
991
991
990
990
989
989
998
988
1007
987
dirt collected 11
1016
986
1025
dirt collected 12
985
dirt collected 13
dirt collected 14
1034
984
1043
983
1052
dirt collected 15
982
1061
981
1070
980
dirt collected 16
1079
dirt collected 17
979
dirt collected 18
1088
978
1097
977
dirt collected 19
1106
dirt collected 20
dirt collected 21
976
1115
975
1124
dirt collected 22
dirt collected 23
974
1133
973
1142
972
dirt collected 24
1151
971
1160
970
1169
969
1178
dirt collected 25
968
1187
dirt collected 26
dirt collected 27
967
dirt collected 28
1196
966
1205
dirt collected 29
965
1214
964
1223
963
1232
962
1241
961
dirt collected 30
1240
960
1239
959
1238
958
dirt collected 31
1237
957
1236
dirt collected 32
956
dirt collected 33
1235
dirt collected 34
955
1234

681
1271
680
1270
679
1269
678
1268
677
1267
676
1266
675
1265
674
1264
673
1263
672
1262
671
1261
670
1260
669
1259
668
1258
667
1257
666
1256
665
1255
664
1254
663
1253
662
1252
661
1251
660
1250
659
1249
658
1248
657
1247
656
1246
655
1245
654
1244
653
1243
652
1242
651
1241
650
1240
649
1239
648
1238
647
1237
646
1236
645
1235
644
1234
643
1233
642
1232
641
1231
640
1230
639
1229
638
1228
637
1227
636
1226
635
1225
634
1224
633
1223
632
1222
631
1221
630
1220
629
1219
628
1218
627
dirt collected 170
dirt collected 171
dirt collected 172
dirt collected 173
1217
626
1216
625
1215
624
1214
623
dirt collected 174
1213
622
1212
621
1211
620
1210
629
1209
638
1208
647
1207
656
1206
665
1205
674
1204
683
1203
692
1202
701
1201
710
1200
719
1199
728
1198
737
1197
746
1196
755
1195
764
1194
773
1193
782
1192
791
1191
800
1190
809
1189
818
1188
827
1187
836
1186
845
1185
854
1184
863
1183
872
1182
881
1181
dirt collected 175
890
1180
899
1179
908
1178
917
1177
926
1176
935
1175
944
1174
953


564
1094
563
1093
562
1092
561
1091
560
1090
559
1089
558
1088
557
1087
556
1086
555
1085
554
1084
553
1083
552
1082
551
1081
550
1080
549
1079
548
1078
547
1077
546
1076
545
1075
544
1074
543
1073
542
1072
541
1071
540
1070
539
1069
538
1068
537
1067
536
1066
535
1065
534
1064
533
1063
532
1062
531
1061
530
1060
529
1059
528
1058
527
1057
526
1056
525
1055
524
1054
523
1053
522
1052
521
1051
520
1050
519
1049
518
1048
517
1047
516
1046
515
dirt collected 260
1045
514
1044
513
1043
512
1042
511
1041
510
1040
509
1039
508
1038
507
1037
506
1036
505
1035
504
1034
503
1033
502
1032
501
1031
500
1030
499
1029
498
1028
497
1027
496
1026
495
1025
494
1024
503
1023
512
1022
521
1021
530
1020
539
1019
548
1018
557
1017
566
1016
575
1015
584
1014
593
1013
602
1012
611
1011
620
1010
629
1009
638
1008
647
1007
656
1006
665
1005
674
1004
683
1003
692
1002
701
1001
710
1000
719
999
728
998
737
997
746
996
755
995
764
994
773
993
782
992
791
991
800
990
809
989
818
988
827
987
836
986
845
985
854
98

930
1320
929
1319
928
1318
927
1317
926
1316
925
1315
924
1314
923
1313
922
1312
921
1311
920
1310
919
1309
918
1308
917
1307
916
1306
915
1305
914
1304
913
1303
912
1302
911
1301
910
1300
909
1299
908
1298
907
1297
906
1306
905
1315
904
1324
903
1333
902
1342
901
1351
900
1360
899
1369
898
1378
897
1387
896
1396
895
1405
894
1414
893
1423
892
1432
891
1441
890
1450
889
1459
888
1468
887
1467
886
1466
885
1465
884
1464
883
1463
882
1462
881
1461
880
1460
879
1459
878
1458
877
1457
876
1456
875
1455
874
1454
873
1453
872
1452
871
1451
870
1450
869
1449
868
1448
867
1447
866
1446
865
1445
864
1444
863
1443
862
1442
861
1441
860
1440
859
1439
858
1438
857
1437
856
1436
855
1435
854
1434
853
1433
852
1432
851
1431
850
1430
849
1429
848
1428
847
1427
846
1426
845
1425
844
1424
843
1423
842
1422
841
1421
840
1420
839
1419
838
1418
837
1417
836
1416
835
1415
834
1414
833
1413
832
1412
831
1411
830
1410
829
1409
828
1408
827
1407
826
1406
825
1405
824
1404
823
1403
822
1402
821
1401
820
1400
8

922
592
921
591
920
590
919
589
918
588
917
587
916
586
915
585
914
584
913
583
912
582
911
581
910
580
909
579
908
578
907
577
906
576
905
575
904
574
903
573
902
572
901
571
900
570
899
569
898
568
897
567
896
566
895
565
894
564
893
563
892
562
891
561
890
560
889
559
888
558
887
557
886
556
885
555
884
554
883
553
882
552
881
551
880
550
879
549
878
548
877
547
876
546
875
545
874
544
873
543
872
542
871
541
870
540
869
539
868
538
867
537
866
546
865
555
864
564
863
573
862
582
861
591
860
600
859
609
858
618
857
627
856
636
855
645
854
654
853
663
852
672
851
681
850
690
849
699
848
708
847
717
846
726
845
735
844
744
843
753
842
762
841
771
840
780
839
789
838
798
837
807
836
816
835
825
834
834
833
843
832
852
831
861
830
870
829
879
828
888
827
897
826
906
825
915
824
924
823
933
822
942
821
951
820
960
819
969
818
978
817
987
816
996
815
1005
814
1014
813
1023
812
1032
811
1041
810
1050
809
1059
808
1068
807
1077
806
1086
805
1095
804
1104
803
1113
802
1122
801
1131
800
1140


...

570
1960
569
1959
568
1958
567
1957
566
1956
565
1955
564
1954
563
1953
562
1952
561
1951
560
1950
559
1949
558
1948
557
1947
556
1946
555
1945
554
1944
553
1943
552
1942
551
1941
550
1940
549
1939
548
1938
547
1937
546
1936
545
1935
544
1934
543
1933
542
1932
541
1931
540
1930
539
1929
538
1928
537
1927
536
1926
535
1925
534
1924
533
1923
532
1922
531
1921
530
1920
529
1919
528
1918
527
1917
526
1916
525
1915
524
1914
533
1913
542
1912
551
1911
560
1910
569
1909
578
1908
587
1907
596
1906
605
1905
614
1904
623
1903
632
1902
641
1901
650
1900
659
1899
668
1898
677
1897
686
1896
695
1895
704
1894
713
1893
722
1892
731
1891
740
1890
749
1889
758
1888
767
1887
776
1886
785
1885
794
1884
803
1883
812
1882
821
1881
830
1880
839
1879
848
1878
857
1877
866
1876
875
1875
884
1874
893
1873
902
1872
911
1871
920
1870
929
1869
938
1868
947
1867
956
1866
965
1865
974
1864
983
1863
992
1862
1001
1861
1010
1860
1019
1859
1028
1858
1037
1857
1046
1856
1055
1855
1064
1854
1073
1853
1082
1852
1091
1851

520
1080
519
1079
518
1078
517
1077
516
1076
515
1075
514
1074
513
1073
512
1072
511
1071
510
1070
509
1069
508
1068
507
1067
506
1066
505
1065
504
1064
503
1063
502
1062
501
1061
500
1060
499
1059
498
1058
497
1057
496
1056
495
1055
494
1054
493
1053
492
1052
491
1051
490
1050
489
1049
488
1048
487
1047
486
1046
485
1045
484
1044
483
1043
482
1042
481
1041
480
1040
479
1039
478
1038
477
1037
476
1036
475
1035
474
1034
473
1033
472
1032
471
1031
470
1030
469
1029
468
1028
467
1027
466
1026
465
1025
464
1024
463
1023
462
1022
461
1021
460
1020
459
1019
458
1018
457
1017
456
1016
455
1015
454
1014
453
1013
452
1012
451
1011
450
1010
449
1009
448
1008
447
1007
446
1006
445
1005
444
1004
443
1003
442
1002
441
1001
440
1000
439
999
438
998
437
997
436
996
435
995
434
994
433
993
432
992
431
991
430
990
429
989
428
988
427
987
426
986
425
985
424
984
423
983
422
982
421
981
420
980
419
979
418
978
417
977
416
976
415
975
414
974
413
973
412
972
411
971
410
970
409
969
408
968
407
967
406
966

712
1042
711
1041
710
1040
709
1039
708
1038
707
1037
706
1036
705
1035
704
1034
703
1033
702
1032
701
1031
700
1030
699
1029
698
1028
697
1027
696
1026
695
1025
694
1024
693
1023
692
1022
691
1021
690
1020
689
1019
688
1018
687
1017
686
1016
685
1015
684
1014
683
1013
682
1012
681
1011
680
1010
679
1009
678
1008
677
1007
676
1006
675
1005
674
1004
673
1003
672
1002
671
1001
670
1000
669
999
668
998
667
997
666
996
665
995
664
994
663
993
662
992
661
991
660
990
659
989
658
988
657
987
656
986
655
985
654
984
653
983
652
982
651
981
650
980
649
979
648
978
647
977
646
976
645
975
644
974
643
973
642
972
641
971
640
970
639
969
638
968
637
967
636
966
635
965
634
964
633
963
632
962
631
961
630
960
629
959
628
958
627
957
626
956
625
955
624
954
623
953
622
952
621
951
620
950
619
949
618
948
617
947
616
946
615
945
614
944
613
943
612
942
611
941
610
940
609
939
608
938
607
937
606
936
605
935
604
934
603
933
602
932
601
931
600
930
599
929
598
928
597
927
596
926
595
925
594
924
593
9

1831
1171
1830
1180
1829
1189
1828
1198
1827
1207
1826
1216
1825
1225
1824
1234
1823
1243
1822
1252
1821
1261
1820
1270
1819
1279
1818
1288
1817
1297
1816
1306
1815
1315
1814
1324
1813
1333
1812
1342
1811
1351
1810
1360
1809
1369
1808
1378
1807
1387
1806
1396
1805
1405
1804
1414
1803
1423
1802
1432
1801
1441
1800
1450
1799
1459
1798
1468
1797
1477
1796
1486
1795
1495
1794
1504
1793
1513
1792
1522
1791
1531
1790
1530
1789
1529
1788
1528
1787
1527
1786
1526
1785
1525
1784
1524
1783
1523
1782
1522
1781
1521
1780
1520
1779
1519
1778
1518
1777
1517
1776
1516
1775
1515
1774
1514
1773
1513
1772
1512
1771
1511
1770
1510
1769
1509
1768
1508
1767
1507
1766
1506
1765
1505
1764
1504
1763
1503
1762
1502
1761
1501
1760
1500
1759
1499
1758
1498
1757
1497
1756
1496
1755
1495
1754
1494
1753
1493
1752
1492
1751
1491
1750
1490
1749
1489
1748
1488
1747
1487
1746
1486
1745
1485
1744
1484
1743
1483
1742
1482
1741
1481
1740
1480
1739
1479
1738
1478
1737
1477
1736
1476
1735
1475
1734
1474
1733
1473
1732
1472


invalid command name "5230692672moveIt"
    while executing
"5230692672moveIt"
    ("after" script)
